In [328]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")

import warnings
warnings.filterwarnings('ignore')

# xcrun: error: invalid active developer path

In [329]:
conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)

In [333]:
transactions = pd.read_sql("SELECT * FROM transactions", db)
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [ ]:
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

In [332]:
transactions.columns[-1]

'senatorId'

In [ ]:
def returnSenatorNameFromId(senId):
    sens = senators.senator.to_list()
    senIds = senators.senatorId.to_list()

    senatorDict = dict(zip(senIds, sens))
    return senatorDict[senId]

returnSenatorNameFromId(103)

In [6]:
def lookupIdByName(nameString):
    nameMatch = senators[senators['senator'].str.contains(nameString)]

    if len(nameMatch)>1:
        return nameMatch
    else: 
        return nameMatch['senatorId']


lookupIdByName("Tuber")

0    100
Name: senatorId, dtype: int64

In [7]:
tub  = transactions[transactions['senatorId']==100]
tub.ticker.value_counts().head(15)


--      88
ECOM    30
CLF     28
X       26
PYPL    24
OXY     20
INTC    19
MSFT    15
SSYS    15
BABA    14
AA      14
CALX    10
QCOM     9
F        8
NU       6
Name: ticker, dtype: int64

In [8]:
tub[(tub['ticker']=='QCOM') & (tub['type']=='Purchase') 
& (tub['asset_type']=='Stock')
]


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
261,06/16/2022,Joint,QCOM,QUALCOMM Incorporated - Common Stock,Stock,Purchase,"$250,001 - $500,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/13/2022,261,100
1007,03/29/2021,Joint,QCOM,QUALCOMM Incorporated,Stock,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/23/2021,1002,100


In [9]:

def reformatDate(transDate):
    listDate  = transDate.split("/")
    return f"{listDate[2]}-{listDate[0]}-{listDate[1]}"

transactions['transaction_date'] = transactions['transaction_date'].apply(lambda row: reformatDate(row))

transactions['transaction_date'].head()


0    2022-10-28
1    2022-10-31
2    2022-10-28
3    2022-10-28
4    2022-10-24
Name: transaction_date, dtype: object

In [10]:
for frame in [historicalPricesAG, historicalPricesHR, historicalPricesSZ]:
    frame['date'] = pd.to_datetime(frame['date'])

qc = historicalPricesHR[['QCOM', 'SPY', 'date']]
temp = qc[qc['date']>= '2021-03-29']
temp


,QCOM,SPY,date
0,120.2000,394.5900,2022-11-21
1,123.8500,396.0300,2022-11-18
2,121.4300,398.5100,2022-11-11
3,106.6900,376.3500,2022-11-04
4,119.2100,389.0200,2022-10-28
...,...,...,...
82,134.8904,408.3881,2021-04-30
83,131.6153,407.8400,2021-04-23
84,134.3170,408.3489,2021-04-16
85,136.6106,402.7022,2021-04-09


In [11]:
temp = temp.sort_values(by="date")
temp.head(10)

,QCOM,SPY,date
86,133.9089,392.0545,2021-04-01
85,136.6106,402.7022,2021-04-09
84,134.3170,408.3489,2021-04-16
83,131.6153,407.8400,2021-04-23
82,134.8904,408.3881,2021-04-30
81,133.9672,413.1052,2021-05-07
80,126.4841,407.6835,2021-05-14
79,127.7572,406.0785,2021-05-21
78,130.7504,411.0696,2021-05-28
77,131.2195,413.5749,2021-06-04


In [12]:


def createGrowthColumn(ticker, frame):

    pricesList = np.array(frame[ticker])
    pctChanges = np.diff(pricesList)/pricesList[:-1]

    currentInvestment , initialInvestment= 100, 100
    investmentValues = []

    for c in pctChanges[1:]: 
        currentInvestment = currentInvestment*(1+c) 
        investmentValues.append(currentInvestment)

    pctChanges = [np.nan ] + list(pctChanges)
    investmentValues = [ np.nan, initialInvestment ] + investmentValues

    invName= ticker+'_growth'
    pctName = ticker+'_pchange'
    frame[invName] = investmentValues
    frame[pctName] = pctChanges



createGrowthColumn('QCOM', temp)
createGrowthColumn('SPY', temp)
temp.head()


,QCOM,SPY,date,QCOM_growth,QCOM_pchange,SPY_growth,SPY_pchange
86,133.9089,392.0545,2021-04-01,NaN,NaN,NaN,NaN
85,136.6106,402.7022,2021-04-09,100.000000,0.020176,100.000000,0.027159
84,134.3170,408.3489,2021-04-16,98.321067,-0.016789,101.402202,0.014022
83,131.6153,407.8400,2021-04-23,96.343402,-0.020114,101.275831,-0.001246
82,134.8904,408.3881,2021-04-30,98.740800,0.024884,101.411937,0.001344


In [13]:
import plotly.express as px 

fig = px.line(temp, x='date', y=['QCOM_growth', 'SPY_growth' ],
 title="Growth in Qualcomm v S&P"
 )


fig.show()

What a great example! looks like there was a sharp rise in price starting in November, and by mid January Qualcomm had grown much more than the S&P. My hypothesis is that the majority of stock purchases will exhibit a similar pattern

In [14]:
import string 
print(string.ascii_uppercase)

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [15]:
def findSourceFrame(ticker):
    leadingLetter =ticker[0]

    relevantCols = ['date', ticker, 'SPY']

    if leadingLetter in 'ABCDEFG':
        rez = historicalPricesAG[relevantCols]
    elif leadingLetter in 'HIJKLMNOPQR':
        rez = historicalPricesHR[relevantCols]
    elif leadingLetter in 'STUVWXYZ':
        rez = historicalPricesSZ[relevantCols]
    else:
        return 'ticker leading letter is non-alpha'
    return rez




def generateHistoricalPricesTable(ticker, date):
    
    rez = findSourceFrame(ticker)

    rez = rez.sort_values(by="date")

    # start dataframe at purchase date 
    rez = rez[rez['date']>=date]
    
    # end dataframe at one year from purchase date 
    if len(rez)>52:
        rez = rez.head(52)

    return rez


graphDims = {'height':500,  'width':800}

def graphGrowthRates(frame, tickerFirst, tickerSecond='SPY'):
    yColumnList = [tickerFirst+"_growth", tickerSecond+"_growth"]
    fig = px.line(frame, x='date', y=yColumnList,
    title=f"Growth in {tickerFirst} v {tickerSecond}",
        height= graphDims['height'] , width= graphDims['width']
    )
    fig.show()


# putting it all together 
def graphAlpha(ticker, date):

    histPrices =  generateHistoricalPricesTable(ticker, date)
    createGrowthColumn(ticker, histPrices)
    createGrowthColumn('SPY', histPrices)

    graphGrowthRates(histPrices, ticker)
 
    


In [17]:

def purchasesBySenator(senId):
    isSenator = transactions['senatorId']==senId
    isPurchase = transactions['type']=='Purchase'
    isStock = transactions['asset_type']=='Stock'
    rez = transactions[isSenator & isPurchase & isStock]

    return rez.loc[:, :'type']


tbPurchases = purchasesBySenator(100)
tbPurchases.tail(10)


,transaction_date,owner,ticker,asset_description,asset_type,type
1034,2021-03-29,Joint,ATHM,Autohome Inc.,Stock,Purchase
1035,2021-03-29,Joint,AME,"AMETEK, Inc.",Stock,Purchase
1036,2021-03-29,Joint,GOOGL,Alphabet Inc.,Stock,Purchase
1039,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
1040,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
1043,2021-03-12,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1056,2021-04-19,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1059,2021-04-16,Joint,ECOM,ChannelAdvisor Corporation,Stock,Purchase
1069,2021-04-07,Joint,OXY,Occidental Petroleum Corporation,Stock,Purchase
1082,2021-05-06,Joint,X,United States Steel Corporation,Stock,Purchase


In [18]:
graphAlpha("OXY", '2021-06-25')

In [19]:
alDate = reformatDate("03/16/2021")
graphAlpha('AA', alDate)


Alright so Tommy Tuberville had a great year in 2021, but the S&P was pretty lousy that year. Let's try a different Senator and another time period

In [20]:
senSample = senators.sample(n=30, random_state=69)['senatorId'].to_list()
# lets find the senator with the most purchases within a random sample of 30 senators 
maxPurchases= 0
maxSenator=0
for sen in senSample:
    currentPurchases = len(purchasesBySenator(sen))
    if currentPurchases >= maxPurchases:
        maxPurchases = currentPurchases
        maxSenator = sen

print(senators[senators['senatorId']==maxSenator])
purchasesBySenator(maxSenator).tail(10)

    row              senator  senatorId       party
34   35  David A Perdue , Jr        136  Republican


,transaction_date,owner,ticker,asset_description,asset_type,type
8382,2015-04-17,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8383,2015-04-23,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8384,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8385,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8387,2015-04-15,Joint,MOS,The Mosaic Company (NYSE),Stock,Purchase
8393,2015-05-12,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8394,2015-05-13,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8395,2015-05-14,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8397,2015-05-15,Joint,AXLL,Axiall Corporation (NYSE),Stock,Purchase
8398,2015-05-29,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase


In [21]:
graphAlpha("MOS", "2015-04-15")
print("this is is kind of a terrible trade")

this is is kind of a terrible trade


In [22]:
graphAlpha("KN", "2015-04-24")
print("this one has a bit of a pop, but also not a great trade")

this one has a bit of a pop, but also not a great trade


In [23]:
purchasesBySenator(136).iloc[-60:-50, :]

,transaction_date,owner,ticker,asset_description,asset_type,type
8284,2015-04-02,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8285,2015-04-02,Joint,AVP,Avon Products Inc. (NYSE),Stock,Purchase
8287,2015-04-02,Joint,JNJ,Johnson & Johnson (NYSE),Stock,Purchase
8288,2015-04-02,Joint,KO,The Coca-Cola Company (NYSE),Stock,Purchase
8290,2015-05-06,Joint,SYY,Sysco Corporation (NYSE),Stock,Purchase
8292,2015-05-11,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8294,2015-05-12,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8295,2015-01-07,Joint,AM,Antero Midstream Partners LP (NYSE),Stock,Purchase
8296,2015-01-14,Joint,AR,Antero Resources Corporation (NYSE),Stock,Purchase
8297,2015-01-15,Joint,BPL,"Buckeye Partners, L.P. (NYSE)",Stock,Purchase


In [24]:
graphAlpha("AM", "2015-01-15")

In [25]:
# purchasesBySenator(136).iloc[-90:-80, :]
purchasesBySenator(136).iloc[20:30]

,transaction_date,owner,ticker,asset_description,asset_type,type
2404,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2406,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2408,2020-03-09,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
2409,2020-03-10,Joint,MSM,"MSC Industrial Direct Co., Inc.",Stock,Purchase
2410,2020-03-10,Joint,M,"Macy's, Inc.",Stock,Purchase
2411,2020-03-10,Joint,NWSA,News Corporation,Stock,Purchase
2412,2020-03-10,Joint,FOXA,Fox Corporation,Stock,Purchase
2413,2020-03-10,Joint,EAF,GrafTech International Ltd.,Stock,Purchase
2415,2020-03-13,Joint,ENB,Enbridge Inc.,Stock,Purchase
2418,2020-03-13,Joint,TRP,TC Energy Corporation,Stock,Purchase


In [26]:
graphAlpha("M", "2020-03-10")

here's a great example of why we need more information. Its true Macy's had a big run, but this run was 6 months after the purchase, and it was preceeded by a smaller run. Hard to say if the first run was the reason for the purcahse or not. Lets modify the graph alpha function to indicate all transacions (purcahses and sales) of one asset by a senator within the measurement window (1 year)

In [27]:
from dateutil.relativedelta import  relativedelta

transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])


def findRelevantTransactions(senId, date, ticker):
    startDate = pd.to_datetime(date)
    endDate = startDate + relativedelta(years=1)
    
    afterStartDate = transactions['transaction_date']>= startDate
    beforeEndDate = transactions['transaction_date']< endDate
    isStock = transactions['asset_type']=="Stock"
    isSenator = transactions['senatorId']==senId
    isTicker = transactions['ticker']==ticker

    rez = transactions[afterStartDate & beforeEndDate & isStock & isSenator & isTicker]

    return rez 



print("looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run ")

macysTransactions = findRelevantTransactions(136, "2020-03-10", "M")[['transaction_date', 'type','amount', 'senator', 'ticker']]
macysTransactions

looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run 


,transaction_date,type,amount,senator,ticker
1980,2020-04-14,Sale (Full),"$15,001 - $50,000","David A Perdue , Jr",M
2410,2020-03-10,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M
2466,2020-03-20,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M


In [28]:
# from datetime import datetime
import datetime
#generate map of dates and transaction types 
dates = macysTransactions['transaction_date'].to_list()
transTypes = macysTransactions['type'].to_list()

#calculate the closest friday to merge with historical prices table (weekly data)
def nearestFriday(inputDate):
    fridayDelta = datetime.timedelta( (4-inputDate.weekday()) % 7 )
    nearestFriday = inputDate + fridayDelta
    nearestFridayString = datetime.datetime.strftime(nearestFriday , "%Y-%m-%d")
    return nearestFridayString

fridayDates  = [nearestFriday(d) for d in dates]

transactionMap = dict(zip(fridayDates, transTypes))



transactionMap 

{'2020-04-17': 'Sale (Full)',
 '2020-03-13': 'Purchase',
 '2020-03-20': 'Purchase'}

In [29]:

# putting it all together
# find relevant transactions
def historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput ):
    relevantTransactions = findRelevantTransactions(senatorIdInput, dateInput,tickerInput)
    # return relevantTransactions

    dates = relevantTransactions['transaction_date'].to_list()
    transTypes = relevantTransactions['type'].to_list()
    fridayDates  = [nearestFriday(d) for d in dates]
    transactionMap = dict(zip(fridayDates, transTypes)) 

    hist = generateHistoricalPricesTable(tickerInput, dateInput)


    hist['date']  = hist['date'].dt.strftime("%Y-%m-%d")

    mappedColumn = []
    isTransaction = []



    for date in hist['date'].to_list():
        if date in transactionMap.keys():
            mappedColumn.append(transactionMap[date])
        else:
            mappedColumn.append(np.nan)
            # mappedColumn.append('fill')

    hist['transactionType'] = mappedColumn
    hist['transactionValue'] = np.where(hist['transactionType'].notnull(), hist[tickerInput], np.nan)


    return hist


# temp = historicalPricesWithTransaction(136,  "2020-03-10", "M" )
# temp.head()


In [30]:
# temp = historicalPricesWithTransaction(125,  "2017-12-21", "WTW" )
temp = historicalPricesWithTransaction(136,  "2020-03-10", "M" )
temp.head()

,date,M,SPY,transactionType,transactionValue
141,2020-03-13,7.7445,257.7998,Purchase,7.7445
140,2020-03-20,5.8060,220.3585,Purchase,5.8060
139,2020-03-27,5.3334,244.0702,NaN,NaN
138,2020-04-03,4.6390,239.0331,NaN,NaN
137,2020-04-09,6.4232,267.9359,NaN,NaN


In [31]:
import plotly.graph_objects as go 

def graphTransactions_fromHistorical(senatorIdInput, frame, tickerInput):


    senatorName = returnSenatorNameFromId(senatorIdInput)


    fig = px.scatter(frame, x="date", y="transactionValue", color="transactionType",
    labels=dict(transactionValue="historical Price "+ tickerInput),
    height= graphDims['height'] , width= graphDims['width'], 
    title = senatorName+ " Historical Price and Transactions "+ tickerInput)

    fig.add_trace(go.Scatter(x=frame['date'], y=frame[tickerInput], mode="lines", line=go.scatter.Line(color="grey"), 
    name=tickerInput+' Historical Price'))



    fig.show()

graphTransactions_fromHistorical(136, temp, "M")



this wasn't the best series of trades. Next let's combine the preceeding couple of functions to examine trade's success by only inputting the transaction date, ticker, and senator. 

In [32]:
# putting it all together 

def graphTransactions_fromSource(senatorIdInput, dateInput,tickerInput):
    historicalPricesFrame = historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput )

    graphTransactions_fromHistorical(senatorIdInput, historicalPricesFrame, tickerInput)



In [90]:

purchases = transactions[(transactions['type']=='Purchase') & (transactions['asset_type']=='Stock') ]
randomPurchase = purchases.sample(n=1, random_state=227)[['transaction_date','ticker','asset_description','senator','senatorId']]
randomPurchase

,transaction_date,ticker,asset_description,senator,senatorId
7244,2016-03-31,WMT,Wal-Mart Stores Inc.,William Cassidy,134


In [91]:
# randomPurchase.ticker[0]
randomticker = randomPurchase.ticker.values[0]
randomDateFull = randomPurchase.transaction_date.values[0]
randomDate = str(randomDateFull).split("T")[0]
randomSenator = randomPurchase.senatorId.values[0]

graphTransactions_fromSource(randomSenator,  randomDate, randomticker)


Looks like Kelly Loeffler's purchase had a pretty big pop after she bought it, but she sold and missed out on a lot of gains. 
Let's graph some more transactions at random

In [45]:
def generateRandomPurchase(randomState):
    purchases = transactions[transactions['type']=='Purchase']
    randomPurchase = purchases.sample(n=1,random_state=randomState)[['transaction_date','ticker','asset_description','senator','senatorId']]


    print(randomPurchase)
    randomticker = randomPurchase.ticker.values[0]
    randomDateFull = randomPurchase.transaction_date.values[0]
    randomDate = str(randomDateFull).split("T")[0]
    randomSenator = randomPurchase.senatorId.values[0]

    return randomSenator,  randomDate, randomticker


def graphRandomPurcahse(randomState):

    randomSenator,  randomDate, randomticker = generateRandomPurchase(randomState)

    graphTransactions_fromSource(randomSenator,  randomDate, randomticker)


graphRandomPurcahse(427)



     transaction_date ticker asset_description              senator  senatorId
3799       2019-02-05      T         AT&T Inc.  David A Perdue , Jr        136


Excellent trades David Perdue! 

In [36]:
graphRandomPurcahse(430)

     transaction_date ticker asset_description      senator  senatorId
1713       2020-07-13   AAPL        Apple Inc.  Ron L Wyden        163


Well done Ronald! Personally, I wouldn't sell either. There's a substantial jump in price, particularly after the first purchase. However, this analsysis is only sound if we differentiate between:  <br>
    &nbsp; changes in the marketplace in general AND <br>
    &nbsp; changes in this stock in particular <br>
Next we build upon previous funcions, graphing all transactions of a single senator in a single stock, but using growth rates instead of just historical prices. 

In [37]:
# pd.to_datetime("2020-03-03")+timedelta(weeks=1)

prevWeek = datetime.datetime.strptime("2020-03-03", "%Y-%m-%d") +datetime.timedelta(weeks=-1)
datetime.datetime.strftime(prevWeek,"%Y-%m-%d")


'2020-02-25'

In [38]:
def historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput ):

    #start dataframe a week early so growth rates begin at 1
    prevWeekDate = datetime.datetime.strptime(dateInput, "%Y-%m-%d") +datetime.timedelta(weeks=-1)
    prevWeekString = datetime.datetime.strftime(prevWeekDate,"%Y-%m-%d")
    

    historicalPricesFrame = historicalPricesWithTransaction(senatorIdInput, prevWeekString,tickerInput )

    createGrowthColumn(tickerInput, historicalPricesFrame)
    createGrowthColumn('SPY', historicalPricesFrame)

     #remove preceeding week from  dataframe 

    historicalPricesFrame=     historicalPricesFrame.iloc[1:, :]

    growthColumn = tickerInput+'_growth'

    historicalPricesFrame['transactionValue'] = np.where(historicalPricesFrame['transactionType'].notnull(),\
         historicalPricesFrame[growthColumn], np.nan)

    return historicalPricesFrame


temp = historicalGrowthRatesWithTransactions(136,  "2020-03-10", "M"  )
temp.head()

,date,M,SPY,transactionType,transactionValue,M_growth,M_pchange,SPY_growth,SPY_pchange
141,2020-03-13,7.7445,257.7998,Purchase,100.000000,100.000000,-0.265388,100.000000,-0.094601
140,2020-03-20,5.8060,220.3585,Purchase,74.969333,74.969333,-0.250307,85.476599,-0.145234
139,2020-03-27,5.3334,244.0702,NaN,NaN,68.866938,-0.081399,94.674317,0.107605
138,2020-04-03,4.6390,239.0331,NaN,NaN,59.900575,-0.130198,92.720437,-0.020638
137,2020-04-09,6.4232,267.9359,NaN,NaN,82.938860,0.384609,103.931772,0.120915


In [67]:
def graphAlphaGrowth_fromHistorical(senatorIdInput, frame, tickerInput):
    senatorName = returnSenatorNameFromId(senatorIdInput)


    fig = px.scatter(frame, x="date", y="transactionValue", color="transactionType",
    labels=dict(transactionValue="Growth Rate"),
    height= graphDims['height'] , width= graphDims['width'], 
    title = senatorName+ " Growth Rates "+ tickerInput)

    growthColumn = tickerInput+'_growth'

    fig.add_trace(go.Scatter(x=frame['date'], y=frame[growthColumn], mode="lines", line=go.scatter.Line(color="#EB4E4B"), 
    name=tickerInput+' Growth Rate'))

    fig.add_trace(go.Scatter(x=frame['date'], y=frame['SPY_growth'], mode="lines", line=go.scatter.Line(color="#349FEB"), 
    name='SPY Growth Rate'))



    fig.show()

graphAlphaGrowth_fromHistorical(136, temp, "M")

In [42]:
def graphAlphaGrowth_fromSource(senatorIdInput, dateInput,tickerInput):
    historical = historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput )
    graphAlphaGrowth_fromHistorical(senatorIdInput, historical , tickerInput)

In [68]:

def graphRandomPurcahse_alpha(randomState):
    randomSenator,  randomDate, randomticker = generateRandomPurchase(randomState)

    graphAlphaGrowth_fromSource(randomSenator,  randomDate, randomticker)

graphRandomPurcahse_alpha(761)

     transaction_date ticker                        asset_description  \
7984       2015-09-16    MMC  Marsh & McLennan Companies, Inc. (NYSE)   

            senator  senatorId  
7984  Gary C Peters        108  


# Finding Measurement Window


In [78]:
randomSenator,  randomDate, randomticker = generateRandomPurchase(456)
findRelevantTransactions(randomSenator,  randomDate, randomticker)

     transaction_date ticker                     asset_description  \
5573       2017-09-08  SGAPY  Singapore Telecommunications Limited   

               senator  senatorId  
5573  Shelley M Capito        110  


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
5234,2017-12-19,Spouse,SGAPY,Singapore Telecommunications Limited,Stock,Sale (Full),"$1,001 - $15,000",--,Shelley M Capito,https://efdsearch.senate.gov/search/view/ptr/a...,01/10/2018,5233,110
5573,2017-09-08,Spouse,SGAPY,Singapore Telecommunications Limited,Stock,Purchase,"$1,001 - $15,000",--,Shelley M Capito,https://efdsearch.senate.gov/search/view/ptr/2...,10/06/2017,5573,110


In [92]:
purchases.sort_values("transaction_date").head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
8458,2012-09-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8458,101
8457,2012-12-17,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8457,101
8456,2013-03-15,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8456,101
8455,2013-06-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8455,101
8454,2013-09-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8454,101


In [99]:
graphAlphaGrowth_fromSource(101, '2012-09-13','DD')


Observe the multiple purcahses of DD made by Thomas Carper in 2012. His first purchase in September did not result in a growth rate more than the market average. 

In [106]:
graphAlphaGrowth_fromSource(101, '2012-12-21','DD')

Starting the clock from his next purcahse in December of 2012 paints a rosier picture. From here, he managed to beat the S&P by a wide margin.  Thomas' first purcahse was poorly timed, but is subsequent purcahse three months later was very profitable. This is why every purcahse needs to be examined separately.
<br>
<br>
the analysis that follows will calculate alpha as the maximum difference in growth indecies between a stock and the S&P 6 months after every purcahse. 

In [186]:
def alphaFrame(senatorIdInput, dateInput,tickerInput ):
    hist = historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput)
    hist.drop(['transactionType', 'transactionValue'], axis=1, inplace=True)

    if(hist.shape[0]>26):
        #keep only 6 months of data, or 26 weeks 
        hist = hist.iloc[:26,:]

    growthCol = f"{tickerInput}_growth"
    #calculate alpha as difference in growth rates 
    hist['alpha']  = hist[growthCol] - hist['SPY_growth']
    return hist

# temp = alphaFrame(101, '2012-12-21','DD')
# temp = alphaFrame(101, '2012-12-21','M')
temp = alphaFrame(101, '2012-12-21','F')

temp

,date,F,SPY,F_growth,F_pchange,SPY_growth,SPY_pchange,alpha
518,2012-12-21,7.8890,119.0519,100.000000,0.068478,100.000000,0.012047,0.000000
517,2012-12-28,8.5608,116.7507,108.515655,0.085157,98.067062,-0.019329,10.448593
516,2013-01-04,9.0264,122.0367,114.417543,0.054387,102.507142,0.045276,11.910402
515,2013-01-11,9.3124,122.6204,118.042844,0.031685,102.997432,0.004783,15.045412
514,2013-01-18,9.3856,123.6709,118.970719,0.007860,103.879820,0.008567,15.090898
513,2013-01-25,9.0996,125.2717,115.345418,-0.030472,105.224444,0.012944,10.120974
512,2013-02-01,8.7234,126.0971,110.576752,-0.041342,105.917755,0.006589,4.658997
511,2013-02-08,8.7770,126.5640,111.256179,0.006144,106.309937,0.003703,4.946242
510,2013-02-15,8.7234,126.8225,110.576752,-0.006107,106.527069,0.002042,4.049683
509,2013-02-22,8.3616,126.6391,105.990620,-0.041475,106.373019,-0.001446,-0.382399


there we have it, 6 months of price data from the date of purchase. In this case, an alpha of 11.35 was achieved several months later

In [202]:
def calculateAlphaValue(senatorIdInput, dateInput,tickerInput ):

    try:         
        # we're looking to return the maximum alpha value within a 6 month period
        histAlpha = alphaFrame(senatorIdInput, dateInput,tickerInput )

    except: 
        return 'Not Found'

        # start counting a week after purcahse
    histAlpha = histAlpha.iloc[ 1:,:]

    maxAlpha = histAlpha.loc[histAlpha.alpha.idxmax()].alpha

    return maxAlpha



15.746535685302746

In [147]:
purchases.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
1,2022-10-31,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,1,100
4,2022-10-24,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,4,100
5,2022-10-13,Joint,NU,Nu Holdings Ltd. Class A Ordinary Shares,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,5,100
6,2022-10-13,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$250,001 - $500,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,6,100
7,2022-10-11,Joint,NU,Nu Holdings Ltd. Class A Ordinary Shares,Stock,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,7,100


In [171]:
timeTest = purchases.transaction_date.values[1]
# datetime.datetime.strftime(timeTest, "%Y-%m-%d")
timeTest
# purchases.ticker.values[3]

numpy.datetime64('2022-10-24T00:00:00.000000000')

In [229]:
alphaList = []

for row in range(len(purchases)):
# for row in range(10):
    currentDate = purchases.transaction_date.values[row]
    currentTicker = purchases.ticker.values[row]
    currentSenId = purchases.senatorId.values[row]


    currentDateString  = str(currentDate).split("T")[0]
   
    try:
        currentAlpha = calculateAlphaValue(currentSenId, currentDateString,currentTicker)
    except:
        currentAlpha ="Not Found"


    

    alphaList.append(currentAlpha)
alphaList[:5]


[10.898455269172317,
 2.325887275806508,
 12.969423298853371,
 1.2743223109707316,
 12.969423298853371]

In [230]:
purchases['alpha'] = alphaList
# purchases.sort_values(by="alpha", ascending=False)
# purchases.head()
purchases['alpha'] = purchases['alpha'].replace("Not Found", -9999)
topAlphaBuys = purchases.sort_values(by="alpha", ascending=False).head(15)
topAlphaBuys[['transaction_date', 'ticker','senatorId','asset_description', 'alpha']]


,transaction_date,ticker,senatorId,asset_description,alpha
1397,2020-11-20,SI,101,Silvergate Capital Corporation,450.026941
5428,2017-10-19,CRSP,132,CRISPR Therapeutics AG,188.417248
1557,2020-08-03,TSLA,133,"Tesla, Inc.",187.982706
1384,2020-11-20,NTLA,101,"Intellia Therapeutics, Inc.",147.030607
2451,2020-03-18,CDLX,136,"Cardlytics, Inc.",123.212113
2452,2020-03-18,CDLX,136,"Cardlytics, Inc.",123.212113
7479,2016-01-08,CLR,132,"Continental Resources, Inc.",115.501927
1624,2020-07-07,TSLA,118,"Tesla, Inc.",109.716997
2595,2020-03-10,HAL,119,Halliburton Company,107.017428
1555,2020-08-17,NVAX,133,"Novavax, Inc.",95.695611


There you have it, the best stock purchases made by senators. Lets graph a few of these to see what they look like

In [231]:
graphAlphaGrowth_fromSource(101, '2020-11-20','SI')

In [232]:
graphAlphaGrowth_fromSource(132, '2017-10-19','CRSP')

# Alpha EDA

In [233]:
#remove fill in values
purchasesFilt = purchases[ purchases['alpha']!= -9999]

#remove top outlier 
purchasesFilt = purchasesFilt[ purchasesFilt['alpha']< 200]



purchasesFilt['alpha'].describe()

count    2832.000000
mean       12.254362
std        16.265922
min       -25.097944
25%         2.384561
50%         7.527119
75%        16.928533
max       188.417248
Name: alpha, dtype: float64

In [238]:
fig = px.histogram(purchasesFilt, x="alpha",
 height= graphDims['height'] , width= graphDims['width'], 
 nbins=50, 
 marginal="box", 
 title="alpha Distribution")
fig.show()

In [257]:
# alphaSer = purchasesFilt.groupby('senatorId')['alpha'].mean()
alphaSer = purchasesFilt.groupby('senator')['alpha'].mean()
alphaSer =alphaSer.sort_values(ascending=False).head(10)
fig = px.bar(alphaSer,
 height= graphDims['height'] , width= graphDims['width'], 
 title="Top Performing Senators", 
 )


fig.update_layout(yaxis_title="Mean Alpha")
fig.show()

In [285]:
amountSer = purchasesFilt.groupby('amount')['alpha'].mean()
amountSer.index

Index(['$1,000,001 - $5,000,000', '$1,001 - $15,000', '$100,001 - $250,000',
       '$15,001 - $50,000', '$25,000,001 - $50,000,000', '$250,001 - $500,000',
       '$5,000,001 - $25,000,000', '$50,001 - $100,000'],
      dtype='object', name='amount')

In [286]:
amountDict = dict(amountSer)

orderedKeys = ['$25,000,001 - $50,000,000', 
    '$5,000,001 - $25,000,000',
'$1,000,001 - $5,000,000', 
'$250,001 - $500,000',
'$100,001 - $250,000',
'$50,001 - $100,000',
'$15,001 - $50,000',
'$1,001 - $15,000'
]

valuesOrdered =  [amountDict[key] for key in orderedKeys]
amountDict_ordered = dict(zip(orderedKeys, valuesOrdered))
amountDict_ordered

{'$25,000,001 - $50,000,000': 0.5898915243843845,
 '$5,000,001 - $25,000,000': 12.67463492213362,
 '$1,000,001 - $5,000,000': 4.333993913729238,
 '$250,001 - $500,000': 8.298416380483207,
 '$100,001 - $250,000': 20.49967038477412,
 '$50,001 - $100,000': 13.982868038749034,
 '$15,001 - $50,000': 13.981950997716048,
 '$1,001 - $15,000': 11.482797420747808}

In [309]:
amountSer_ordered = pd.Series(amountDict_ordered)
amountFrame_ordered = pd.DataFrame(amountSer_ordered)
amountFrame_ordered.columns =["mean alpha"]

fig = px.bar(amountFrame_ordered)

fig.update_layout(xaxis_title="purchase value", yaxis_title="mean alpha", title="Mean Alpha by Purchase Amount")

fig.show()



In [312]:
countSer = purchasesFilt.groupby('amount')['alpha'].count()
countDict = dict(countSer)
valuesOrdered =  [countDict [key] for key in orderedKeys]
countDict_ordered = dict(zip(orderedKeys, valuesOrdered))


countSer_ordered = pd.Series(countDict_ordered)
countFrame_ordered = pd.DataFrame(countSer_ordered)
countFrame_ordered.columns =["count"]

fig = px.bar(countFrame_ordered)

fig.update_layout(xaxis_title="purchase value", yaxis_title="count", title="Count by Purchase Amount")

fig.show()


In [320]:
senators[senators['senator'].str.contains('Mitch')]

,row,senator,senatorId,party
5,6,"A. Mitchell Mcconnell, Jr.",105,Republican


In [325]:
purchasesFilt[purchasesFilt['senatorId']==105].sort_values(by="transaction_date").head(2)

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
4740,2017-09-07,Spouse,WFC,Wells Fargo & Company,Stock,Purchase,"$1,001 - $15,000",Dividend Reinvestment,"A. Mitchell Mcconnell, Jr.",https://efdsearch.senate.gov/search/view/ptr/0...,05/15/2018,4740,105,18.061525
4739,2017-12-05,Spouse,WFC,Wells Fargo & Company,Stock,Purchase,"$1,001 - $15,000",Dividend Reinvestment,"A. Mitchell Mcconnell, Jr.",https://efdsearch.senate.gov/search/view/ptr/0...,05/15/2018,4739,105,4.398816


In [323]:
graphAlphaGrowth_fromSource(105, '2017-09-07','WFC')

# Post alpha (from purchases) to db

In [374]:
purchases.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8751,2014-12-10,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8749,106,0.329482
8752,2015-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8750,106,0.427035
8753,2014-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8751,106,0.338179
8760,2014-12-19,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8758,133,23.900003
8762,2014-12-11,Spouse,MIK,"The Michaels Companies, Inc. (NASDAQ)",Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8760,133,22.137520


In [400]:



def findFullDates(frame):
    frame['deleteMe'] = frame['transaction_date'].astype(str)
    fullDates = frame[frame['deleteMe'].str.len() > 10].shape[0]
    print('fullDates', fullDates)

    frame.drop('deleteMe', axis=1, inplace=True)

findFullDates(purchases)


fullDates 0


In [401]:

findFullDates(transactions)



fullDates 0


In [ ]:
transactions.head()

In [393]:
t = transactions[transactions['type']!= 'Purchase']
t['alpha'] = np.nan
t.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
0,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,0,100,NaN
2,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,2,100,NaN
3,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,3,100,NaN
8,2022-10-06,Joint,XOM,Exxon Mobil Corporation Common Stock Option Ty...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,8,100,NaN
9,2022-10-06,Joint,CVX,Chevron Corporation Common Stock Option Type: ...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,9,100,NaN


In [394]:
t.type.value_counts()

Sale (Full)       2525
Sale (Partial)    1879
Unknown            467
Exchange           102
Name: type, dtype: int64

In [397]:
print('t Length', t.shape[0])
print('purchases Length', purchases.shape[0])

t Length 4973
purchases Length 3420


In [398]:
transactions_new = pd.concat([t, purchases])
transactions_new.shape


(8393, 14)

In [402]:

findFullDates(transactions_new)

fullDates 3420


looks like its interpreting teh date as a date when i concat

In [ ]:
transactions_new.tail()

In [415]:
timeTestEnd = transactions_new.tail()['transaction_date'].values[0]
timeTestEnd

Timestamp('2014-12-10 00:00:00')

In [417]:
timeTestStart = transactions_new['transaction_date'][0]
timeTestStart

'2022-10-28'

In [419]:
for test in [timeTestStart, timeTestEnd]:
    test = str(test)
    print(test.split(" "))

['2022-10-28']
['2014-12-10', '00:00:00']


In [420]:
def extractDateFromTimestamp(row):
    return str(row).split(" ")[0]

transactions_new['transaction_date'] = transactions_new['transaction_date'].apply(lambda row: extractDateFromTimestamp(row))
transactions_new.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8751,2014-12-10,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8749,106,0.329482
8752,2015-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8750,106,0.427035
8753,2014-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8751,106,0.338179
8760,2014-12-19,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8758,133,23.900003
8762,2014-12-11,Spouse,MIK,"The Michaels Companies, Inc. (NASDAQ)",Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8760,133,22.137520


In [371]:
transactions_new[transactions_new['alpha'].isnull()].type.value_counts()

Sale (Full)       2525
Sale (Partial)    1879
Unknown            467
Exchange           102
Name: type, dtype: int64

In [372]:
transactions_new.alpha.value_counts()

-9999.000000    587
 23.187226        8
 3.859860         8
 16.923277        8
 6.241248         6
               ... 
 19.243351        1
 6.069588         1
-1.076865         1
 9.288799         1
 22.137520        1
Name: alpha, Length: 2274, dtype: int64

In [432]:

transactions_new['alpha'].replace({-9999:"Ticker Not Found"}, inplace=True)
numAlpha =  transactions_new[transactions_new['alpha'].notnull()]

numAlpha.alpha.value_counts().values.min(), numAlpha.alpha.value_counts().values.max()


(1, 587)

In [ ]:
# transactions_new.to_csv("./CSV Files/transactions_new.csv")

In [434]:

import psycopg2 

conn=None
cur=None 
db = None

def postDataFrameDB():
    try:
        conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
        db =  create_engine(url=conn_string)
        conn=db.connect()

        transactions_new.to_sql('transactions', con=conn, if_exists='replace', index=False)

        connPsy = psycopg2.connect(conn_string)
        cur = connPsy.cursor()
        connPsy.autocommit = True

        sqlString = """
        select * from transactions limit 5;
        """
        cur.execute(sqlString)

        records = cur.fetchall()
        for row in records:
            print(row)

    except Exception as err:
        print(err)
    finally: 
        if cur is not None: 
            cur.close()
        if conn is not None: 
            conn.close()
    
postDataFrameDB()


('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 0, 100, None)
('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 2, 100, None)
('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 3, 100, None